In [ ]:
#preprocessing steps
import os
import cv2
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
def img2data(filepath,img_size):
    #changing to current directory
    label_dict={}
    labels=os.listdir(filepath)  #extracting the sub-folders from main folder
    data=[]
    target=[]
    for i in range(len(labels)):
        label_dict[labels[i]]=i
    del labels[0]
    print(labels)
    for label in labels:
        img_names=os.listdir(os.path.join(filepath,label))  #Extracting images from the sub-folder
        for img_name in img_names:
          try:
            img=cv2.imread(os.path.join(filepath,label,img_name)) #Reading the image in pixel format
            new_img=cv2.resize(img,(img_size,img_size))#Resizing image to required size
            new_img=preprocess_input(new_img) #preprocessing the images
            data.append(new_img) #Appending the resized image to data list
            target.append(label_dict[label])
          except:
            print('a') 
        print('b') #Appending the appropriate folder value in which the image present
    data=np.array(data)#Normalizing the image pixels to be in range of [0,1]
    target=np.array(target) #specifing target label values
    np.save('data1',data)
    np.save('target1',target)
img2data("./train",128)#Function call with parameters as ('filepath','required_image_size')

['outdoor', 'indoor']
b
a
a
a
a
a
a
a
a
b


In [ ]:
import shutil
folders = os.listdir('/content/train/indoor')
for i in folders:
  path = os.path.join('/content/train/indoor', i)
  images = os.listdir(path)
  for i in images:
    src = os.path.join(path, i)
    dest = '/content/train/indoor'
    shutil.copy(src,'/content/train/indoor')
#check image copies out side the folders, if works properly remove breaks and run 

In [ ]:
#after this delete all folders using
for i in folders:
  path = os.path.join('/content/train/indoor', i)
  shutil.rmtree(path)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model,model_from_json
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import tensorflow as tf

In [ ]:
import numpy as np
from keras.utils import np_utils
X=np.load('./data1.npy')
tar=np.load('./target1.npy')
for i in range(len(tar)):
  if tar[i] == 2:
    tar[i] = 0
y=np_utils.to_categorical(tar,num_classes=2)

In [ ]:
#splitting the dataset
(trainX, testX, trainY, testY) = train_test_split(X,y,
	test_size=0.20, stratify=y, random_state=42)

# construct the training image generator for data augmentation

In [ ]:
INIT_LR = 1e-4 #initial learning rate
EPOCHS = 20 #number of epochs
BS = 32 #batch_size
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(128,128,3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = tf.keras.layers.GlobalAveragePooling2D()(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


In [ ]:
print("[INFO] training head...")
H = model.fit(trainX, trainY, batch_size=BS,
	validation_data=(testX, testY),
	epochs=EPOCHS)

[INFO] training head...
Epoch 1/20
256/256 [==============================] - 5s 21ms/step - loss: 0.2439 - accuracy: 0.9021 - val_loss: 0.0533 - val_accuracy: 0.9814
Epoch 2/20
256/256 [==============================] - 5s 18ms/step - loss: 0.0625 - accuracy: 0.9783 - val_loss: 0.0384 - val_accuracy: 0.9873
Epoch 3/20
256/256 [==============================] - 5s 18ms/step - loss: 0.0472 - accuracy: 0.9836 - val_loss: 0.0334 - val_accuracy: 0.9878
Epoch 4/20
256/256 [==============================] - 4s 18ms/step - loss: 0.0365 - accuracy: 0.9881 - val_loss: 0.0308 - val_accuracy: 0.9878
Epoch 5/20
256/256 [==============================] - 4s 18ms/step - loss: 0.0276 - accuracy: 0.9918 - val_loss: 0.0296 - val_accuracy: 0.9887
Epoch 6/20
256/256 [==============================] - 4s 18ms/step - loss: 0.0231 - accuracy: 0.9927 - val_loss: 0.0283 - val_accuracy: 0.9873
Epoch 7/20
256/256 [==============================] - 5s 18ms/step - loss: 0.0213 - accuracy: 0.9927 - val_loss: 0.029

In [ ]:
model.save('./narmodel.h5')

In [ ]:
model1=tf.keras.models.load_model('narmodel.h5')
model1.evaluate(testX,testY)

64/64 [==============================] - 1s 14ms/step - loss: 0.0222 - accuracy: 0.9912


[0.02221566066145897, 0.9911937117576599]